# Stock News Sentiment and Relevance
With some scraping magic, our team collected news headlines for our `FANG` Stocks from `https://www.nasdaq.com/symbol/{ticker}/news-headlines` into `dataset/nasdaq/FANG.csv`

This notebook shows how we compute the **News Sentiments** and **Relavance** for a given stock for the purpose of doing studies against stock returns

## Let's Load Our Data

In [2]:
import pandas as pd

In [4]:
fname = 'dataset/nasdaq/raw.csv'
data = pd.read_csv( fname, header = 0, index_col = 0)

In [5]:
data.tail()

,stockcode,source,headline,article,urls
datetime,,,,,
"February 04, 2019, 03:26:00 PM EDT",NFLX,Zacks.com,"Super Bowl Spending, Streaming & Amazon's Big ...",\n Welcome to the latest episode of the Fu...,https://www.nasdaq.com/article/super-bowl-spen...
"February 04, 2019, 01:24:00 PM EDT",NFLX,Zacks.com,Buy Apple (AAPL) Stock on Streaming TV Push to...,\n Shares of Apple AAPL jumped over 2.5% Mond...,https://www.nasdaq.com/article/buy-apple-aapl-...
"February 04, 2019, 12:42:43 PM EDT",NFLX,InvestorPlace Media,3 Must-Watch Comic Book Movies on Netflix,"\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/3-must-watch-co...
"February 04, 2019, 12:36:05 PM EDT",NFLX,InvestorPlace Media,2 Crazy Acquisitions That Would Boost Alphabet...,"\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/2-crazy-acquisi...
"February 04, 2019, 04:38:00 PM EDT",NFLX,NaN,US STOCKS-Boost in tech shares sends Wall Stre...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/us-stocksboost-...


In [6]:
data['timestring'] = data.index
data['timestring'] = data['timestring'].astype('str')
data['tz'] = data['timestring'].apply( lambda x: x[-3:])

In [7]:
data[data['tz']!= 'EDT']

,stockcode,source,headline,article,urls,timestring,tz
datetime,,,,,,,
NaN,AMZN,NaN,"The Zacks Analyst Blog Highlights: Netflix, Di...",\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/the-zacks-analy...,nan,nan
NaN,AMZN,NaN,Amazon Announces New Job Positions for New Yor...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/amazon-announce...,nan,nan
NaN,AMZN,NaN,Why Amazon (AMZN) is Poised to Beat Earnings E...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/why-amazon-amzn...,nan,nan
NaN,AMZN,NaN,Amazon (AMZN) Dips More Than Broader Markets: ...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/amazon-amzn-dip...,nan,nan
NaN,GOOGL,NaN,Amazon Announces New Job Positions for New Yor...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/amazon-announce...,nan,nan
NaN,GOOGL,NaN,Alphabet (GOOGL) Dips More Than Broader Market...,\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/alphabet-googl-...,nan,nan
NaN,NFLX,NaN,"The Zacks Analyst Blog Highlights: Netflix, Di...",\n\n\nShutterstock photo\n\n\n\n\n\n\n\r\n ...,https://www.nasdaq.com/article/the-zacks-analy...,nan,nan


### Remove Bad Data

In [8]:
dup_filter = data.duplicated()
na_dt = data.index.isna()

print(f'Dataset has {len(data)} points, of which; {sum(dup_filter)} are duplicates, and {sum(na_dt)} are missing DateTime info.')

Dataset has 6405 points, of which; 384 are duplicates, and 7 are missing DateTime info.


In [9]:
data = data.drop_duplicates()
data = data[ ~ data.index.isna()]
print(f'After cleaning we have {len(data)} data points')

After cleaning we have 6014 data points


In [10]:
data['tz'] = data['tz'].apply( lambda x : 'EST' if x == 'EDT' else x)
data['dt'] = pd.to_datetime(data['timestring']).dt.tz_localize('EST')

/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/dateutil/parser/_parser.py:1206: UnknownTimezoneWarning: tzname EDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  category=UnknownTimezoneWarning)


In [11]:
data.head()

,stockcode,source,headline,article,urls,timestring,tz,dt
datetime,,,,,,,,
"February 07, 2019, 09:11:00 PM EDT",AMZN,NaN,Amazon CEO Jeff Bezos Accuses National Enquire...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/amazon-ceo-jeff...,"February 07, 2019, 09:11:00 PM EDT",EST,2019-02-07 21:11:00-05:00
"February 07, 2019, 02:49:22 PM EDT",AMZN,BNK Invest,"Notable Thursday Option Activity: AMZN, BKNG, ...",\nAmong the underlying components of the S&P 5...,https://www.nasdaq.com/article/notable-thursda...,"February 07, 2019, 02:49:22 PM EDT",EST,2019-02-07 14:49:22-05:00
"February 07, 2019, 06:55:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer owner tr...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,"February 07, 2019, 06:55:00 PM EDT",EST,2019-02-07 18:55:00-05:00
"February 07, 2019, 06:26:00 PM EDT",AMZN,NaN,Amazon's Bezos says National Enquirer tried to...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,"February 07, 2019, 06:26:00 PM EDT",EST,2019-02-07 18:26:00-05:00
"February 07, 2019, 11:55:45 AM EDT",AMZN,InvestorPlace Media,"IRS Tax Refund 2019: So, Where’s My Tax Refund?","\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/irs-tax-refund-...,"February 07, 2019, 11:55:45 AM EDT",EST,2019-02-07 11:55:45-05:00


## Example of Relevancy Source and Sentiment Score assignment

In [12]:
example_no = 57
test_url = data['urls'][example_no]
test_article = data['article'][example_no]

### 1. Text Preprocessing

In [13]:
test_article

'\nInvestorPlace - Stock Market News, Stock Advice & Trading TipsLeading the Apple \xa0(NASDAQ: AAPL ) rumor mill today is news of the company losing an AR team member. Today, we\'ll look at that and other Apple Rumors for Tuesday.makeArticleAd(); Hololens :\xa0 Avi Bar-Zeev is no longer working at Apple , reports Variety . The co-inventor of the Hololens AR headset stopped working for AAPL in January. He notes that his departure was the "best" one could have and wouldn\'t speak about product plans at the company.\xa0Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020. It\'s unknown if that will still take place following his departure.iOS 12.2 Public Beta : The second iOS 12.2 public beta will be coming out today , MacRumors noted. This release will bring public beta testers up to speed with developers. The second developer beta of iOS 12.2 came out yesterday. One of the noteworthy additions of this new update is the inclusion of 

In [14]:
def CleanText(inText):
    import re
    outText = re.sub(r"(\n)|(\r)|('\')", '', inText)
    outText = outText.replace(u'\xa0',' ').encode('utf-8')
    outText = outText.decode('utf-8')
    
    return outText

In [15]:
test_article_clean = CleanText(test_article)
test_article_clean

'InvestorPlace - Stock Market News, Stock Advice & Trading TipsLeading the Apple  (NASDAQ: AAPL ) rumor mill today is news of the company losing an AR team member. Today, we\'ll look at that and other Apple Rumors for Tuesday.makeArticleAd(); Hololens :  Avi Bar-Zeev is no longer working at Apple , reports Variety . The co-inventor of the Hololens AR headset stopped working for AAPL in January. He notes that his departure was the "best" one could have and wouldn\'t speak about product plans at the company. Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020. It\'s unknown if that will still take place following his departure.iOS 12.2 Public Beta : The second iOS 12.2 public beta will be coming out today , MacRumors noted. This release will bring public beta testers up to speed with developers. The second developer beta of iOS 12.2 came out yesterday. One of the noteworthy additions of this new update is the inclusion of four new An

In [18]:
def FilterOutCssJs(x):
    import re
    x = re.sub(r'Read the original article.+','', x)
    x = re.sub(r'Shutterstock.+photo\@media.+\(Reuters\) \-','',x)
    x = re.sub(r'<\^.+', ' ', x)
    x = re.sub(r'max.+important;}}','',x)
    x = re.sub(r'=======================================.+','',x)
    x = re.sub(r'\(?function\(\).+', '', x)
    x = re.sub(r'\(Reporting.+', '', x)
    x = re.sub(r'makeAd.+\;', '', x)
    x = re.sub(r'Source: Shutterstock', '',x)
    x = re.sub(r'Reuters@media.+ \-', '', x)

    return x

In [19]:
FilterOutCssJs(test_article_clean)

'InvestorPlace - Stock Market News, Stock Advice & Trading TipsLeading the Apple  (NASDAQ: AAPL ) rumor mill today is news of the company losing an AR team member. Today, we\'ll look at that and other Apple Rumors for Tuesday.makeArticleAd(); Hololens :  Avi Bar-Zeev is no longer working at Apple , reports Variety . The co-inventor of the Hololens AR headset stopped working for AAPL in January. He notes that his departure was the "best" one could have and wouldn\'t speak about product plans at the company. Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020. It\'s unknown if that will still take place following his departure.iOS 12.2 Public Beta : The second iOS 12.2 public beta will be coming out today , MacRumors noted. This release will bring public beta testers up to speed with developers. The second developer beta of iOS 12.2 came out yesterday. One of the noteworthy additions of this new update is the inclusion of four new An

### 2. Get Article Summary

In [16]:
from summa import summarizer
summary_summa = summarizer.summarize(test_article_clean, ratio = 0.3)
summary_summa

"Today, we'll look at that and other Apple Rumors for Tuesday.makeArticleAd(); Hololens :  Avi Bar-Zeev is no longer working at Apple , reports Variety .\nAvi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020.\nThis has it trailing well behind Amazon's (NASDAQ: AMZN ) Echo line with its 70% share of the market and Alphabet's (NASDAQ: GOOG , GOOGL ) Google Home with its 24% share of the market.Check out more recent Apple Rumors or Subscribe to Apple Rumors : RSSAs of this writing, William White did not hold a position in any of the aforementioned securities.Compare BrokersThe post Tuesday Apple Rumors: Hololens Co-Inventor Avi Bar-Zeev Leaves Apple appeared first on InvestorPlace ."

In [17]:
from sumy.summarizers.lsa import LsaSummarizer
from sumy.nlp.stemmers import Stemmer
from sumy.nlp.tokenizers import Tokenizer
from sumy.utils import get_stop_words
from sumy.parsers.plaintext import PlaintextParser

lang = 'english'
sent_count = 5

parser = PlaintextParser.from_string(test_article_clean, Tokenizer(lang))
LsaSum = LsaSummarizer( Stemmer(lang))
LsaSum.stop_words = get_stop_words(lang)

summary_sumy = LsaSum(parser.document, sent_count)

for isent in summary_sumy:
    print(isent)

InvestorPlace - Stock Market News, Stock Advice & Trading TipsLeading the Apple  (NASDAQ: AAPL ) rumor mill today is news of the company losing an AR team member.
Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020.
One of the noteworthy additions of this new update is the inclusion of four new Animoji for users to play around with.HomePod : The HomePod isn't doing all that well in the U.S. , reports 9to5Mac .
A recent study by CIRP claims that the HomePod currently only holds 6% of the smartspeaker market in the U.S.
This has it trailing well behind Amazon's (NASDAQ: AMZN ) Echo line with its 70% share of the market and Alphabet's (NASDAQ: GOOG , GOOGL ) Google Home with its 24% share of the market.Check out more recent Apple Rumors or Subscribe to Apple Rumors : RSSAs of this writing, William White did not hold a position in any of the aforementioned securities.Compare BrokersThe post Tuesday Apple Rumors: Hololens Co-Inventor Av

### 3. Relavency Score

In [18]:
import spacy

# requirement:
# Run this in your command prompt:
# $ python -m spacy download en

nlp = spacy.load('en_core_web_sm')
doc = nlp(summary_summa)

for ent in doc.ents:
    print(f'{ent.text} <- {ent.label_}')

Today <- DATE
Apple <- ORG
Rumors <- PRODUCT
Hololens <- PERSON
Avi Bar-Zeev <- PERSON
Apple <- ORG
Variety <- GPE

 <- GPE
Avi Bar-Zeev <- PERSON
AR <- ORG
AAPL <- ORG
2020 <- DATE

 <- GPE
Amazon <- ORG
NASDAQ <- ORG
Echo <- GPE
70% <- PERCENT
Alphabet <- ORG
NASDAQ <- ORG
Google Home <- ORG
24% <- PERCENT
Apple <- ORG
Subscribe <- ORG
Apple <- ORG
William White <- PERSON
Compare BrokersThe <- PERSON
Tuesday <- DATE
Apple <- ORG
Hololens Co-Inventor <- ORG
Avi Bar-Zeev <- PERSON
Apple <- ORG
first <- ORDINAL
InvestorPlace <- ORG


In [19]:
AmazonKw = ['AMZN', 'Amazon', 'Jeff Bezos', 'Bezos', 
            'AWS', 'Amazon Web Services',
            'Amazon Kindle', 'Kindle', 
            'Amazon Echo','Echo', 
            'Amazon Prime', 'Prime', 
            'Alexa',
            'Whole Foods Market', 'Whole Foods',
            'Audible', 'Amazon Studios', 'Goodreads', 'Woot', 'Zappos'
           ]

def GetRelavency(inText, kwList):
    import spacy
    nlp = spacy.load('en_core_web_sm')
    doc = nlp(inText)
    
    rSource = 0
    for ent in doc.ents:
        if ent.text.lower() in [kw.lower() for kw in kwList]:
            rSource += 1
    
    return rSource

In [49]:
GetRelavency(summary_summa, AmazonKw)

2

### 4. Get Sentiment

#### Using Stock Market Lexicon

In [58]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from nltk import sent_tokenize

SIA = SentimentIntensityAnalyzer()
sent_text = sent_tokenize( summary_summa)

sent_count = 0
sent_sum = 0
for sent in sent_text:
    sent_sum += SIA.polarity_scores(sent)['compound']
    sent_count += 1

SIA_Score = sent_sum/ sent_count

print( f'--- Text ---\n{summary_summa}\n--- Sentiment Score: {SIA_Score} ---')

--- Text ---
Today, we'll look at that and other Apple Rumors for Tuesday.makeArticleAd(); Hololens :  Avi Bar-Zeev is no longer working at Apple , reports Variety .
Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020.
This has it trailing well behind Amazon's (NASDAQ: AMZN ) Echo line with its 70% share of the market and Alphabet's (NASDAQ: GOOG , GOOGL ) Google Home with its 24% share of the market.Check out more recent Apple Rumors or Subscribe to Apple Rumors : RSSAs of this writing, William White did not hold a position in any of the aforementioned securities.Compare BrokersThe post Tuesday Apple Rumors: Hololens Co-Inventor Avi Bar-Zeev Leaves Apple appeared first on InvestorPlace .
--- Sentiment Score: 0.12483333333333334 ---


#### Using regular Vader

In [55]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import sent_tokenize

SIA_ = SentimentIntensityAnalyzer()
sent_text = sent_tokenize( summary_summa)

sent_count = 0
sent_sum = 0
for sent in sent_text:
    sent_sum += SIA_.polarity_scores(sent)['compound']
    sent_count += 1

SIA_Score = sent_sum/ sent_count

print( f'--- Text ---\n{summary_summa}\n--- Sentiment Score: {SIA_Score} ---')

--- Text ---
Today, we'll look at that and other Apple Rumors for Tuesday.makeArticleAd(); Hololens :  Avi Bar-Zeev is no longer working at Apple , reports Variety .
Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020.
This has it trailing well behind Amazon's (NASDAQ: AMZN ) Echo line with its 70% share of the market and Alphabet's (NASDAQ: GOOG , GOOGL ) Google Home with its 24% share of the market.Check out more recent Apple Rumors or Subscribe to Apple Rumors : RSSAs of this writing, William White did not hold a position in any of the aforementioned securities.Compare BrokersThe post Tuesday Apple Rumors: Hololens Co-Inventor Avi Bar-Zeev Leaves Apple appeared first on InvestorPlace .
--- Sentiment Score: 0.12483333333333334 ---


#### Side Note: `Newspaper` is pretty cool as well

In [52]:
from newspaper import Article

article = Article(test_url)
article.download()
article.parse()
article.nlp()
article.summary
article.text

'InvestorPlace - Stock Market News, Stock Advice & Trading Tips\n\nLeading the Apple (NASDAQ: AAPL ) rumor mill today is news of the company losing an AR team member. Today, we\'ll look at that and other Apple Rumors for Tuesday.\n\nHololens : Avi Bar-Zeev is no longer working at Apple , reports Variety . The co-inventor of the Hololens AR headset stopped working for AAPL in January. He notes that his departure was the "best" one could have and wouldn\'t speak about product plans at the company. Avi Bar-Zeev was reportedly working on an AR headset for AAPL that some were expecting to come out in 2020. It\'s unknown if that will still take place following his departure.\n\niOS 12.2 Public Beta : The second iOS 12.2 public beta will be coming out today , MacRumors noted. This release will bring public beta testers up to speed with developers. The second developer beta of iOS 12.2 came out yesterday. One of the noteworthy additions of this new update is the inclusion of four new Animoji f

## Let's Test the `sentiment_utils` we wrote

In [25]:
from random import randint
example_no = randint(0, len(data))
test_url = data['urls'][example_no]
test_article = data['article'][example_no]
test_hl = data['headline'][example_no]
ticker = data['stockcode'][example_no]

from sentiment_utils import GetCleanText, GetSummary, GetRelavency, GetSentimentScore, Stock_KW_Dict

In [26]:
# 1. Clean
article_ = GetCleanText(test_article)

# 2. Summarize
summary_ = GetSummary( article_ , method = 'sumy-lex_rank')

# 3. Relavency
relavence_ = GetRelavency( summary_, Stock_KW_Dict[ticker], debugmode = True)

# 4. Sentiment
sentiment_ = GetSentimentScore( summary_, method = 'bespoke')

print(f'--- Article Summary: {example_no} {test_hl}---\n{summary_}\n--- Relavency to {ticker}: {relavence_} ---\n--- Sentiment: {sentiment_} ---')

Found facebook relavent.
--- Article Summary: 4051 7 Big Tech Stocks to End This Bear Market---
Fortunately, fundamentals imply that Big Tech stocks are re-emerging as leaders and beginning to power a big market rally.While near-term optics in the market and among the tech stocks are poor, the long-term fundamentals underlying the tech sector remain promising. At its core, Facebook operates four of the most widely used social media apps in the world, all of which have over 1 billion monthly active users. As such, since early October, Nvidia stock has dropped 50%. Moreover, concerns regarding a global economic slowdown have called into question the health of Netflix's international growth narrative.Altogether, Netflix stock hasn't been itself over the past few months. This robust user growth coupled with price hikes will allow the company to report significant revenue and earnings growth, the likes of which should power Netflix stock higher from its currently depressed base.
--- Relaven

## Get News Source if not availble from Data Scraper

In [40]:
from nltk import sent_tokenize

def get_source(df, debugmode = False):
    """input: the entire dataframe"""

    for i in range(len(df)):
        
        if debugmode:
            print(f'row {i} of {len(df)}')
            
        if pd.isna(df.source[i]):
            sent_tokens = sent_tokenize(df.article[i])

            if len(sent_tokens) > 1:
                if ('RTTNews' in sent_tokens[0]) or ('RTTNews' in sent_tokens[-1]) or ('RTTNews' in sent_tokens[1]):
                    df.source[i] = 'RTTNews'
                elif ('Reuters' in sent_tokens[0]) or ('reuters' in sent_tokens[-1]) or ('Reuters' in sent_tokens[1]):
                    df.source[i] = 'Reuters'
            else:
                if 'RTTNews' in sent_tokens[0]:
                    df.source[i] = 'RTTNews'
                elif 'Reuters' in sent_tokens[0]:
                    df.source[i] = 'Reuters'
    return df

In [42]:
out_df = get_source(data[:1000])
out_df.head()

/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/JHO/.virtualenvs/VADER/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,stockcode,source,headline,article,urls,timestring,tz,dt
datetime,,,,,,,,
"February 07, 2019, 09:11:00 PM EDT",AMZN,RTTNews,Amazon CEO Jeff Bezos Accuses National Enquire...,\n\n\nShutterstock photo\n\n@media screen and ...,https://www.nasdaq.com/article/amazon-ceo-jeff...,"February 07, 2019, 09:11:00 PM EDT",EST,2019-02-07 21:11:00-05:00
"February 07, 2019, 02:49:22 PM EDT",AMZN,BNK Invest,"Notable Thursday Option Activity: AMZN, BKNG, ...",\nAmong the underlying components of the S&P 5...,https://www.nasdaq.com/article/notable-thursda...,"February 07, 2019, 02:49:22 PM EDT",EST,2019-02-07 14:49:22-05:00
"February 07, 2019, 06:55:00 PM EDT",AMZN,Reuters,Amazon's Bezos says National Enquirer owner tr...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,"February 07, 2019, 06:55:00 PM EDT",EST,2019-02-07 18:55:00-05:00
"February 07, 2019, 06:26:00 PM EDT",AMZN,Reuters,Amazon's Bezos says National Enquirer tried to...,\n\n\nReuters\n\n@media screen and (max-device...,https://www.nasdaq.com/article/amazons-bezos-s...,"February 07, 2019, 06:26:00 PM EDT",EST,2019-02-07 18:26:00-05:00
"February 07, 2019, 11:55:45 AM EDT",AMZN,InvestorPlace Media,"IRS Tax Refund 2019: So, Where’s My Tax Refund?","\nInvestorPlace - Stock Market News, Stock Adv...",https://www.nasdaq.com/article/irs-tax-refund-...,"February 07, 2019, 11:55:45 AM EDT",EST,2019-02-07 11:55:45-05:00
